# Dependencies

In [1]:
!git clone https://github.com/hiaac-nlp/CAPIVARA

Cloning into 'CAPIVARA'...
remote: Enumerating objects: 3393, done.
remote: Counting objects: 100% (786/786), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 3393 (delta 540), reused 755 (delta 521), pack-reused 2607
Receiving objects: 100% (3393/3393), 34.24 MiB | 18.91 MiB/s, done.
Resolving deltas: 100% (2459/2459), done.


In [2]:
%cd CAPIVARA/clip_pt
!pip install -r requirements.txt -q
%cd src

/content/CAPIVARA/clip_pt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [3]:
import torch
import numpy as np
import torchvision
from tqdm import tqdm
import torch.nn.functional as F

from models.open_CLIP import OpenCLIP
from models.open_CLIP_adapter import OpenCLIPAdapter
from models.open_clip_wrapper import OpenCLIPWrapper
from utils.capivara_utils import download_pretrained_from_hf

# Load Model

In [4]:
batch_size = 100
open_clip = True
adapter = None # or hiaac-nlp/CAPIVARA-LoRA
num_workers = 2 # colab has a limitation on the number of workers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(">>>>>>> Loading model")
if open_clip:
    if adapter is None:
        model_path = download_pretrained_from_hf(model_id="hiaac-nlp/CAPIVARA")
        model = OpenCLIPWrapper.load_from_checkpoint(model_path, strict=False).model
    else:
        model = OpenCLIPAdapter(inference=True, devices=device)
        model.load_adapters(pretrained_adapter=True, model_path=adapter)
else:
    model = OpenCLIP()

model.eval()
model.to("cuda")

vision_processor = model.image_preprocessor
text_tokenizer = model.text_tokenizer

>>>>>>> Loading model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2kmj76gi.ckpt:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.2.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../root/.cache/huggingface/hub/models--hiaac-nlp--CAPIVARA/snapshots/726ff2b2b39c409a92e71383f79e4bf639aa96d1/2kmj76gi.ckpt`
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


open_clip_pytorch_model.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

# Load CIFAR100

In [6]:
testset = torchvision.datasets.CIFAR100(
    root="./data",
    train=False,
    download=True,
    transform=vision_processor
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

cifar100_classes = [
    "maçã",
    "peixe_de_aquário",
    "bebê",
    "urso",
    "castor",
    "cama",
    "abelha",
    "besouro",
    "bicicleta",
    "garrafa",
    "tigela",
    "menino",
    "ponte",
    "ônibus",
    "borboleta",
    "camelo",
    "lata",
    "castelo",
    "lagarta",
    "gado",
    "cadeira",
    "chimpanzé",
    "relógio",
    "nuvem",
    "barata",
    "sofá",
    "caranguejo",
    "crocodilo",
    "xícara",
    "dinossauro",
    " golfinho",
    "elefante",
    "peixe chato",
    "floresta",
    "raposa",
    "menina",
    "hamster",
    "casa",
    "canguru",
    "teclado",
    "lâmpada",
    "cortador de grama",
    "leopardo",
    "leão",
    "lagarto",
    "lagosta",
    "homem",
    "árvore de bordo",
    "motocicleta",
    "montanha",
    "rato",
    "cogumelo",
    "árvore de carvalho",
    "laranja",
    "orquídea",
    "lontra",
    "árvore de palma",
    "pera",
    "caminhonete",
    "pinheiro",
    "planície",
    "prato",
    "papoula",
    "ouriço",
    "gamba",
    "coelho",
    "guaxinim",
    "arraia",
    "estrada",
    "foguete",
    "rosa",
    "mar",
    "foca",
    "tubarão",
    "musaranho",
    "mofeta",
    "arranha-céu",
    "caracol",
    "cobra",
    "aranha",
    "esquilo",
    "bonde",
    "girassol",
    "pimentão doce",
    "mesa",
    "tanque",
    "telefone",
    "televisão",
    "tigre",
    "trator",
    "trem",
    "truta",
    "tulipa",
    "tartaruga",
    "guarda-roupa",
    "baleia",
    "árvore de salgueiro",
    "lobo",
    "mulher",
    "verme"
]

cifar100_templates = [
    "uma foto de um {}.",
    "uma foto borrada de um {}.",
    "uma foto em preto e branco de um {}.",
    "uma foto de baixo contraste de um {}.",
    "uma foto de alto contraste de um {}.",
    "uma foto ruim de um {}.",
    "uma boa foto de um {}.",
    "uma foto de um pequeno {}.",
    "Uma foto de um grande {}.",
    "uma foto do {}.",
    "uma foto borrada do {}.",
    "uma foto em preto e branco do {}.",
    "uma foto de baixo contraste do {}.",
    "uma foto de alto contraste do {}.",
    "uma foto ruim do {}.",
    "uma boa foto do {}.",
    "uma foto do pequeno {}.",
    "uma foto do grande {}.",
]

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


# Evaluation

In [13]:
def zeroshot_classifier(classnames, templates, model):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] # format with class
            texts = text_tokenizer(texts).cuda() # tokenize
            class_embeddings = model.encode_text(texts) # embed with text encoder
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]


def evaluate(testloader, zeroshot_weights, model):
    targets = []
    preds = []
    confidences = []
    complete_confidences = []
    with torch.no_grad():
        top1, top5, n = 0., 0., 0.
        for i, (images, target) in enumerate(tqdm(testloader)):
            images = images.cuda()
            target = target.cuda()

            # predict
            image_features = model.encode_visual(images)
            logits = 100. * image_features @ zeroshot_weights

            # measure accuracy
            acc1, acc5 = accuracy(logits, target, topk=(1, 5))
            top1 += acc1
            top5 += acc5
            n += images.size(0)

            pred_prob = F.softmax(logits, dim=1)
            highest_prob = torch.max(pred_prob, dim=1)[0]

            preds.extend(torch.argmax(pred_prob, dim=1).cpu().numpy())
            targets.extend(target.cpu().numpy())
            confidences.extend(highest_prob.cpu().numpy())
            complete_confidences.extend(pred_prob)

    top1 = (top1 / n) * 100
    top5 = (top5 / n) * 100

    return top1, top5

In [14]:
zeroshot_weights = zeroshot_classifier(cifar100_classes, cifar100_templates, model)

100%|██████████| 100/100 [00:07<00:00, 14.07it/s]


In [15]:
top1, top5 = evaluate(testloader, zeroshot_weights, model)

print(f"\n\n Top-1 accuracy: {top1:.4f}")
print(f"\n Top-5 accuracy: {top5:.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
<ipython-input-13-ca8048898d5f>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]
100%|██████████| 100/100 [00:32<00:00,  3.08it/s]



 Top-1 accuracy: 70.9200

 Top-5 accuracy: 90.2500
